In [1]:
few_shot_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.train.short.csv'
data_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.smalltest.csv'
output_dir = '/scratch/dzhang5/LLM/TWEET-FID/test-results-autolabel'
model_name = "gpt-3.5-turbo"
label_column = 'sentence_class'
text_column = 'tweet'
explanation_column = 'sentence_explanation'
few_shot_num = 8
few_shot_selection = "semantic_similarity"
verify_few_shot_selection = "semantic_similarity"
use_current_explanation = False
use_ground_explanation = False
token_path = "/home/dzhang5/.cache/huggingface/token" 
cache=False
console_output=True
temperature=0.1
random_shuffle_examples = True
random_shuffle_examples_seed = 1

# Generation Stage

In [2]:
from autolabel.schema import ModelProvider, TaskType
from autolabel.models import register_model, MODEL_REGISTRY
from hf_pipeline_new import HFPipelineLLMNew
from few_shot_new import NewAutoLabelConfig, NewExampleSelectorFactory
from autolabel.few_shot import ExampleSelectorFactory
from template_inst import update_inst_mode
from named_entity_recognition_new import NewNamedEntityRecognitionTask
from classification_new import NewClassificationTask
from autolabel.tasks import TASK_TYPE_TO_IMPLEMENTATION 
import sys

In [3]:
update_inst_mode(model_name)
TASK_TYPE_TO_IMPLEMENTATION[TaskType.NAMED_ENTITY_RECOGNITION] = NewNamedEntityRecognitionTask
TASK_TYPE_TO_IMPLEMENTATION[TaskType.CLASSIFICATION] = NewClassificationTask
sys.modules['autolabel.labeler'].ExampleSelectorFactory = NewExampleSelectorFactory
register_model(ModelProvider.HUGGINGFACE_PIPELINE, HFPipelineLLMNew)

In [4]:
from autolabel import LabelingAgent, AutolabelDataset
import json
import os
import pandas as pd

In [5]:
with open(token_path) as tfile:
    token_str = tfile.read()

from huggingface_hub import login
login(token=token_str)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dzhang5/.cache/huggingface/token
Login successful


In [6]:
no_auto = ["microsoft/prophetnet", "microsoft/phi-2", "google/pegasus-x"]
if any([pre in model_name for pre in no_auto]):
    device_map = None
else:
    device_map = "auto"

In [7]:
refuel_models = ["refuel-llm", "llama-13b-chat"]
if model_name in refuel_models:
    provider = "refuel"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetClassification_{few_shot_selection}_{model_name}"
elif model_name.startswith('gpt'):
    provider = "openai"
    em_provider = "openai"
    model_params = {"max_tokens":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetClassification_{few_shot_selection}_{model_name}"
else:
    provider = "huggingface_pipeline"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature,
                    "quantize": 16, "device_map": device_map,
                    "token": token_str}
    task_name = f"FoodborneIllnessIncidentTweetClassification_{few_shot_selection}_{model_name.split('/')[1]}"

In [8]:
if not os.path.exists(output_dir):
    # Create the directory
    os.makedirs(output_dir)
output_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_' + '_cur_' + str(use_current_explanation) + '_ground_' + str(use_ground_explanation) + '_' + os.path.split(data_path)[-1]
output_path = os.path.join(output_dir, output_name)

In [9]:
test_data = pd.read_csv(data_path)

In [10]:
if use_current_explanation and not use_ground_explanation:
    explanation_column = explanation_column + "_prediction"

# Verification Stage

In [11]:
last_results = pd.read_csv(output_path)

In [30]:
from utils import generate_tweet_verify_ans, generate_tweet_verify_ref, construct_sen_results, generate_word_explanation_prediction
import random

In [13]:
verify_question_column = f'sentence_class_verify_question'
verify_explanation_column = f'sentence_class_verify_explanation'
verify_answer_column = f'sentence_class_verify_answer'
verify_example_selection_label_column = f'sentence_class_verify_example_label'

In [14]:
test_data[verify_question_column] = last_results[f'{label_column}_label'].apply(generate_tweet_verify_ref)
verify_few_shot_data = pd.read_csv(few_shot_path)
random.seed(1)
verify_few_shot_data[[verify_question_column, verify_explanation_column, verify_answer_column, verify_example_selection_label_column]] = verify_few_shot_data[[label_column]].apply(lambda x: generate_tweet_verify_ans(x[label_column]), axis=1, result_type="expand")
verify_few_shot_data.to_csv(few_shot_path, index=False)

In [15]:
last_results["sentence_class_label"].unique()

array(['Yes'], dtype=object)

In [16]:
verify_task_guideline = (f"You are an expert at identifying foodborne illness incident information."
                         '''your task is to verify if the prediction about "whether the given text describes a potential foodborne illness event." is correct or not. '''
                         "Use the following examples as a guide for your analysis and format your responses similarly.")
question = f'''Do you think the prediction "{{{verify_question_column}}}" for the given text is correct or not?'''

In [17]:
if verify_few_shot_selection == few_shot_selection:
    output_verify_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_' + '_cur_' + str(use_current_explanation) + '_ground_' + str(use_ground_explanation) + '_verify_COT_' + os.path.split(data_path)[-1]
    output_final_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_' + '_cur_' + str(use_current_explanation) + '_ground_' + str(use_ground_explanation) + '_final_COT_' + os.path.split(data_path)[-1]
else:
    output_verify_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_' + '_cur_' + str(use_current_explanation) + '_ground_' + str(use_ground_explanation) + '_verify_COT_' + verify_few_shot_selection + '_' + os.path.split(data_path)[-1]
    output_final_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_' + '_cur_' + str(use_current_explanation) + '_ground_' + str(use_ground_explanation) + '_final_COT_' + verify_few_shot_selection + '_' + os.path.split(data_path)[-1]
output_verify_path = os.path.join(output_dir, output_verify_name)
output_final_path = os.path.join(output_dir, output_final_name)

In [18]:
config = {
    "task_name": task_name+'_verification',
    "task_type": "classification",
    "dataset": {
        "label_column": verify_answer_column,
        "text_column": text_column,
        "explanation_column": verify_explanation_column,
        "example_selection_label_column": verify_example_selection_label_column,
        "delimiter": ",",
    },
    "model": {
        "provider": provider,
        "name": model_name,
        "params": model_params
    },
    "embedding": {
        "provider": em_provider,
    },
    "prompt": {
        "task_guidelines": verify_task_guideline,
        "output_guidelines": ('''Your answer will consist of an explanation, followed by the correct answer ("correct" or "not correct").'''
                              f'''Please answer with "correct" if the given prediction is correct, otherwise answer with "not correct".'''
                              '''The last line of the response should always be JSON format with one key: {"label": "the correct answer"}.\n'''
                             ),
        "labels": [
            "correct",
            "not correct"
        ],
        "example_selection_labels":[
            "CorrectYes",
            "CorrectNo",
            "NotCorrectYes",
            "NotCorrectNo",
        ],
        "few_shot_examples": few_shot_path,
        "few_shot_selection": verify_few_shot_selection,
        "few_shot_num": few_shot_num,
        "use_current_explanation": use_current_explanation,
        "random_shuffle_examples": random_shuffle_examples,
        "random_shuffle_examples_seed": random_shuffle_examples_seed,
        "example_template": f"Context: {{{text_column}}}\nQuestion: {question}\nAnswer: Let's think step by step.\n{{{verify_explanation_column}}}\n{{{verify_answer_column}}}",
        "chain_of_thought": True
    }
}

config = NewAutoLabelConfig(config)

In [19]:
verify_few_shot_data =  AutolabelDataset(verify_few_shot_data, config=config)

In [20]:
agent = LabelingAgent(config=config, console_output=console_output, cache=cache)

2024-03-28 06:06:59 autolabel.labeler WARNING: cache parameter is deprecated and will be removed soon. Please use generation_cache and transform_cache instead.
/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [21]:
verify_ds = AutolabelDataset(test_data[[text_column, verify_question_column]], config=config)

In [22]:
agent.plan(verify_ds)

/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
2024-03-28 06:07:03 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 06:07:07 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 06:07:11 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Output()

2024-03-28 06:07:13 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 06:07:13 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 06:07:14 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 06:07:14 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 06:07:14 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 06:07:14 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 06:07:14 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 06:07:14 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 06:07:14 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 06:07:15 httpx INFO: HTTP Request: 

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $0.0947 │
│ Number of Examples       │ 10      │
│ Average cost per example │ $0.0095 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at identifying foodborne illness incident information.your task is to verify if the prediction 
about "whether the given text describes a potential foodborne illness event." is correct or not. Use the following 
examples as a guide for your analysis and format your responses similarly.

Your answer will consist of an explanation, followed by the correct answer ("correct" or "not correct").Please 
answer with "correct" if the given prediction is correct, otherwise answer with "not correct".The last line of the 
response should always be JSON format with one key: {"label": "the correct answer"}.


Some examples with their output answers are provided below:

Context: @USER LOL I know! I've had a stomach bug or food poisoning since yesterday afternoon
Question: Do you think the prediction "The text indicates a foodborne illness incident. " for the given text is 
correct or not?
Answer: Let's think step by step.
This answer is correct.I also think this text indicates a foodborne illness incident.
{"label": "correct"}
Context: I've had food poisoning all day and I'm just miserable
Question: Do you think the prediction "The text indicates a foodborne illness incident. " for the given text is 
correct or not?
Answer: Let's think step by step.
This answer is correct.I also think this text indicates a foodborne illness incident.
{"label": "correct"}
Context: @USER Awww :( I hope it clears up. I just got food poisoning I think ~ throwing up all morning :/
Question: Do you think the prediction "The text indicates a foodborne illness incident. " for the given text is 
correct or not?
Answer: Let's think step by step.
This answer is correct.I also think this text indicates a foodborne illness incident.
{"label": "correct"}
Context: I think i have food poisoning hehe so fun xoxo
Question: Do you think the prediction "The text indicates a foodborne illness incident. " for the given text is 
correct or not?
Answer: Let's think step by step.
This answer is correct.I also think this text indicates a foodborne illness incident.
{"label": "correct"}
Context: @USER @USER I'm literally sick to my stomach I have no words.
Question: Do you think the prediction "The text indicates a foodborne illness incident. " for the given text is 
correct or not?
Answer: Let's think step by step.
This answer is not correct.Instead, I think this text does not indicate a foodborne illness incident.
{"label": "not correct"}
Context: @USER That shit is nasty and I got food poisoning
Question: Do you think the prediction "The text indicates a foodborne illness incident. " for the given text is 
correct or not?
Answer: Let's think step by step.
This answer is correct.I also think this text indicates a foodborne illness incident.
{"label": "correct"}
Context: @USER hope you get food poisoning
Question: Do you think the prediction "The text indicates a foodborne illness incident. " for the given text is 
correct or not?
Answer: Let's think step by step.
This answer is not correct.Instead, I think this text does not indicate a foodborne illness incident.
{"label": "not correct"}
Context: I feel like poooo #headache #stomachache #Fml
Question: Do you think the prediction "The text indicates a foodborne illness incident. " for the given text is 
correct or not?
Answer: Let's think step by step.
This answer is not correct.Instead, I think this text does not indicate a foodborne illness incident.
{"label": "not correct"}

Now I want you to label the following example:
Context: @USER As much fun as I can. Woke up with food poisoning or stomach flu. Been bugging me all day #tmi 
Almost done driving for the day
Question: Do you think the prediction "The text indicates a foodborne illness incident. " for the given text is 
correct or not?
Answer: Let's think step by step.

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [23]:
# now, do the actual labeling
verify_ds = agent.run(verify_ds)

Output()

2024-03-28 06:07:15 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 06:07:16 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-28 06:07:16 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 06:07:17 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-28 06:07:17 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 06:07:17 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-28 06:07:17 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 06:07:19 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-28 06:07:19 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 06:07:21 ht

Actual Cost: 0.0133

In [24]:
new_results = last_results.copy()
new_results[f'{verify_answer_column}_label'] = verify_ds.df[f'{verify_answer_column}_label']

In [25]:
verify_ds.df.to_csv(output_verify_path, index=False)
verify_ds.df.to_pickle(output_verify_path.replace('.csv', '.pkl'))

In [26]:
new_results[f'{label_column}_label'] = new_results[[f'{label_column}_label', f'{verify_answer_column}_label']].apply(lambda x: construct_sen_results(x[f'{label_column}_label'], x[f'{verify_answer_column}_label']), axis=1)

In [31]:
new_results['word_check_reference'] = new_results[f'{label_column}_label'].apply(generate_word_explanation_prediction)

In [28]:
new_results.to_csv(output_final_path, index=False)
new_results.to_pickle(output_final_path.replace('.csv', '.pkl'))